# 程序说明

自动填写消防喷林系统检验批报告。部分表格需要补充填写现场实际信息，该表格运行后将只完成基本内容的填写，其余部分仍需自行补充完善。

In [ ]:
#导入需要的基本族库
import xlwings as xw
import numpy as np
import pandas as pd
import datetime as dt
import time
import sys
import os


#### 需要在指定目录下放置文件“` 喷淋检验批数据.xlsx `”，并将模板表格放在该目录下的“` 标准表格 `”文件夹内。

#### 表格` 喷淋检验批数据.xlsx `的内容如下：

* 工作表1，命名为"**可调信息**"

| 区域划分   | AAA地下1层消防喷淋管道 | AAA1层消防喷淋管道   | AAA2层消防喷淋管道   | AAA3层消防喷淋管道   |
| ------ | ------------- | ------------- | ------------- | ------------- |
| 检验批容量  | 800米          | 1200米         | 500米          | 500米          |
| 水压试验日期 | 2020/9/21     | 2020/9/21     | 2020/9/21     | 2020/9/21     |
| 施工图编号  | 21-001，22-002 | 21-002，22-002 | 21-003，22-002 | 21-004，22-002 |
| 材质     | 热浸锌镀锌钢管       | 热浸锌镀锌钢管       | 热浸锌镀锌钢管       | 热浸锌镀锌钢管       |
| 规格     | DN25~DN150    | DN25~DN150    | DN25~DN150    | DN25~DN150    |
| 是否隐蔽   | 否             | 是             | 是             | 否             |
| 工程名称   | CCC           | CCC           | CCC           | CCC           |
| 结构层数   | EEE           | EEE           | EEE           | EEE           |


* 工作表2，命名为“**基本信息**”

| 待填数据    | 基本信息      |
| ------- | --------- |
| 建设单位    | AAA       |
| 监理单位    | BBB       |
| 施工单位    | CCC       |
| 项目经理    | DDD       |
| 专业工长    | EEE       |
| 工程地点    | FFF       |
| 工作压力    | 0.95      |
| 强度试验压力  | 1.45      |
| 试压泵生产厂  | GGG       |
| 规格型号    | HHH       |
| 仪表检验证号  | III       |
| 仪表编号及精度 | JJJ       |
| 校验有效日期  | KKK       |
| 试验介质    | 水         |
| 分部工程    | 建筑给水排水及供暖 |
| 分项工程    | 消防喷淋系统安装  |
| 工作介质    | 水         |


In [ ]:
folder_path=input('请输入基本信息所在的文件夹路径：')
docu_file = str(folder_path+os.sep +'喷淋检验批数据.xlsx')#基础数据所在表格
docu_list = pd.read_excel(docu_file)

In [ ]:
docu_area = input('本次填写的检验批内容：') #调用基础数据里面的信息
if docu_area in docu_list.columns:
    basic_info = pd.read_excel(docu_file).set_index("区域划分")[docu_area].to_dict()#默认从第一个工作簿中获取数据
    basic_info.update(pd.read_excel(docu_file,sheet_name = '基本信息').set_index("待填数据")["基本信息"].to_dict())
    basic_info['工序内容'] = docu_area
else:
    print('程序运行失败')

In [ ]:
basic_info#用于检查此次运行结果是否满足要求

# B.1.4 工程质量报验表

如果需要指定表格编号，可在下面#处进行修改

In [ ]:
app = xw.App(visible=False,add_book=False)
wb0 = app.books.open(str(folder_path +'\\标准表格\\B.1.4 工程质量报验表.xls'))
sheet0 = wb0.sheets[0]


sheet0.range('I5').value = basic_info["工程名称"]
sheet0.range('E7').value = basic_info["监理单位"]
sheet0.range('K8').value = basic_info["工序内容"]+"及配件安装"
#表格的编号可在此进行指定
#sheet0.range('AS5').value = 表格编号
#sheet0.range('BA5').value = 表格编号
wb0.save(str(folder_path+"\\工序报验\\%s - 00工程质量报验表.xls") % basic_info["工序内容"])
wb0.close()
app.quit()

# 隐蔽资料

- **SN2.4.1阀门的水压试验记录**：由于阀门用于整个项目，因此只需要做一次

###  SN2.4.2 管道水压试验记录

- 1、《建筑给水排水及采暖工程施工质量验收规范》GB50242-2002第4.2.1条：

        ①金属及复合管给水管道系统在试验压力下观测10min，压力降不应大于0．02MPa，然后降到工作压力进行检查，应不渗不漏；

        ②塑料管给水系统应在试验压力下稳压1h，压力降不得超过0.05MPa，然后在工作压力的1.15倍状态下稳压2h，压力降不得超过0．03MPa，同时检查各连接处不得渗漏。

- 2、《消防给水及消火栓系统技术规范》GB50974-2014第12.4.3条：

      水压强度试验的测试点应设在系统管网的最低点。对管网注水时，应将管网内的空气排净，并应缓慢升压，达到试验压力后，稳压30min后，管网应无泄漏、无变形，且压力降不应大于0.05MPa。

- 3、《自动喷水灭火系统施工及验收规范》GB50261-2017第6.2.2条：

      水压强度试验的测试点应设在系统管网的最低点。对管网注水时应将管网内的空气排净，并应缓慢升压，达到试验压力后稳压30min后，管网应无泄漏、无变形，且压力降不应大于0.05MPa。

In [ ]:
#水压试验的资料
pressure_date = basic_info['水压试验日期']
#管道冲洗时间
flushing_date = pressure_date + dt.timedelta(days = 1)

#设置强度试验开始时间
initial_time = dt.timedelta(hours=8,minutes=30, seconds=0)

#强度试验与严密性试验间隔
span_time = dt.timedelta(hours = 3,minutes=30)

#强度试验持续时间
pressure_duration = dt.timedelta(hours = 2, minutes = 0)

#严密性试验持续时间
leak_duration = dt.timedelta(hours = 24) #days = 1

#计算公式
pressure_start = pressure_date + initial_time
pressure_end = pressure_start + pressure_duration
leak_start = pressure_end + span_time
leak_end = leak_start + leak_duration

#其他数据
pressure_result = { "强度试验压力降" : 0 , "严密性试验压力降": 0 ,"试验情况":"各连接处无渗漏"}
pressure_result["施工单位检测结果"]="    管道水压试验符合设计及《建筑给水排水及采暖工程施工质量验收规范》（GB50242-2002）规定要求，试验合格。"

**代入水压试验的信息填写表格**

注意：需在目录下创建“工序报验”建立文件夹

In [ ]:
# 工作簿
app = xw.App(visible=False,add_book=False)
wb1 = app.books.open(str(folder_path +'\\标准表格\\SN2.4.2 给水管道水压试验记录.xls'))
sheet1 = wb1.sheets[0]

#表头信息
sheet1.range('C3').value = basic_info["工程名称"]
sheet1.range('H3').value = basic_info["分项工程"]
sheet1.range('C4').value = basic_info["建设单位"]
sheet1.range('H4').value = basic_info["监理单位"]
sheet1.range('C5').value = basic_info["施工单位"]
sheet1.range('G5').value = basic_info["专业工长"]
sheet1.range('J5').value = pressure_date.strftime('%Y-%m-%d')
sheet1.range('C6').value = basic_info["试压泵生产厂"]
sheet1.range('G6').value = basic_info["规格型号"]
sheet1.range('J6').value = basic_info["试验介质"]
sheet1.range('C7').value = basic_info["仪表检验证号"]
sheet1.range('G7').value = basic_info["仪表编号及精度"]
sheet1.range('J7').value = basic_info["校验有效日期"]
sheet1.range('A10').value = basic_info["工序内容"]

#强度试验
sheet1.range('B10').value = "强度试验"
sheet1.range('C10').value = basic_info["工作压力"]
sheet1.range('D10').value = basic_info["强度试验压力"]
sheet1.range('E10').value = pressure_start.strftime('%Y.%m.%d %H:%M')
sheet1.range('F10').value = pressure_end.strftime('%Y.%m.%d %H:%M')

#严密性试验
sheet1.range('c10').value = "严密性试验"
sheet1.range('C11').value = basic_info["工作压力"]
sheet1.range('D11').value = basic_info["工作压力"]
sheet1.range('E11').value = leak_start.strftime('%Y.%m.%d %H:%M')
sheet1.range('F11').value = leak_end.strftime('%Y.%m.%d %H:%M')

#其他信息
sheet1.range('G10').value = pressure_result["强度试验压力降"]
sheet1.range('G11').value = pressure_result["严密性试验压力降"]
sheet1.range('K10').value = pressure_result["试验情况"]
sheet1.range('K11').value = pressure_result["试验情况"]
sheet1.range('C26').value = pressure_result["施工单位检测结果"]

wb1.save(str(folder_path+"\\工序报验\\%s - 02给水管道水压试验记录.xls") % basic_info["工序内容"])
wb1.close()
app.quit()

### SN2.5.1 管道隐蔽验收记录

In [ ]:
# 隐蔽验收表格需要指定的内容
concealed_info = {"隐蔽工程验收内容和设计及规范要求":
"隐蔽内容及部位：精装吊顶区域及管井内喷淋管道安装符合设计及规范要求；根据图纸所有阀门安装到位，管道支架安装符合设计规范，管道压力试验符合设计规范及施工质量验收规范"}
concealed_info['施工质量检查记录'] = "安装方式、位置均符合设计及规范要求"

In [ ]:

# 工作簿
app = xw.App(visible=False,add_book=False)
wb2 = app.books.open(str(folder_path +'\\标准表格\\SN2.5.1 管道隐蔽验收记录.xls'))
sheet2 = wb2.sheets[0]

#
sheet2.range('E4').value = basic_info["工程名称"]
sheet2.range('P4').value = basic_info["工程地点"]
sheet2.range('E5').value = basic_info["施工单位"]
sheet2.range('P5').value = basic_info["项目经理"]
sheet2.range('U5').value = basic_info["专业工长"]
sheet2.range('E6').value = "/"#分包单位
sheet2.range('P6').value = "/"#分包负责人
sheet2.range('U6').value = "/"#分包单位专业工长
sheet2.range('E7').value = basic_info["分部工程"]
sheet2.range('P7').value = basic_info["分项工程"]
sheet2.range('E8').value = basic_info["工序内容"]+"隐蔽工程"
sheet2.range('P8').value = basic_info["施工图编号"]
sheet2.range('E9').value = concealed_info["隐蔽工程验收内容和设计及规范要求"]
sheet2.range('B14').value = basic_info["工序内容"]
sheet2.range('E14').value = "管道使用%s（%s），符合要求" % (basic_info['材质'],basic_info['规格'])
sheet2.range('O14').value = concealed_info["施工质量检查记录"]

if basic_info["是否隐蔽"] == "是":
    wb2.save(str(folder_path+"\\工序报验\\%s - 03管道隐蔽验收记录.xls") % basic_info["工序内容"])
    wb2.close()
else:
        wb2.close()
app.quit()

**注明** 隐蔽验收部位的信息可能还有没填完的内容，日后需进行检查

# SN2.6.1 管道支、吊架安装记录

存在问题：不同规格管道使用的支架形式不同，因此需要根据现场实际进行修改

In [ ]:
wb3 = xw.Book(str(folder_path +'\\标准表格\\SN2.6.1  管道支、吊架安装记录.xls'))
sheet3 = wb3.sheets[0]
#
sheet3.range('C1').value = basic_info["分项工程"]
sheet3.range('C3').value = basic_info["工程名称"]
sheet3.range('L3').value = basic_info["建设单位"]
sheet3.range('C4').value = basic_info["监理单位"]
sheet3.range('L4').value = basic_info["施工单位"]
sheet3.range('C5').value = basic_info["分项工程"]
sheet3.range('L5').value = basic_info["工序内容"]

if basic_info["是否隐蔽"] == "是":
    wb3.save(str(folder_path+"\\工序报验\\%s - 04管道支、吊架安装记录-待修改.xls") % basic_info["工序内容"])
else:
        wb3.save(str(folder_path+"\\工序报验\\%s - 03管道支、吊架安装记录-待修改.xls") % basic_info["工序内容"])
 #wb3.close()

#管道支、吊架安装记录的信息需要根据实际填写，故表格不关闭



# SN2.7.1 管道系统清（冲）洗记录

In [ ]:
concealed_info["冲洗标准"] = "以系统最大设计流量进行冲洗，直至出水口的水色透明度与目测一致为合格"
concealed_info["标准依据"] = "《建筑给水排水及采暖工程施工质量验收规范》GB50242-2016及设计要求"
concealed_info["冲洗结果记录"] = '出水口水质清澈透明不浑浊，与进水水质相同'


In [ ]:
wb4 = xw.Book(str(folder_path +'\\标准表格\\SN2.7.1 管道系统清（冲）洗记录.xls'))
sheet4 = wb4.sheets[0]

sheet4.range('G2').value = basic_info["分项工程"]
sheet4.range('G5').value = basic_info["工程名称"]
sheet4.range('V5').value = basic_info["建设单位"]
sheet4.range('G6').value = basic_info["监理单位"]
sheet4.range('V6').value = basic_info["施工单位"]
sheet4.range('G7').value = basic_info["工序内容"]
sheet4.range('V7').value = flushing_date.strftime('%Y.%m.%d')
sheet4.range('B9').value = basic_info["材质"]
sheet4.range('L9').value = basic_info["规格"]
sheet4.range('V9').value = basic_info["工作介质"]
sheet4.range('G10').value = concealed_info["冲洗标准"]
sheet4.range('G11').value = concealed_info["标准依据"]
sheet4.range('B13').value = basic_info["工序内容"]
sheet4.range('R13').value = concealed_info["冲洗结果记录"]

if basic_info["是否隐蔽"] == "是":
    wb4.save(str(folder_path+"\\工序报验\\%s - 05通水试验记录.xls") % basic_info["工序内容"])
else:
        wb4.save(str(folder_path+"\\工序报验\\%s - 04通水试验记录.xls") % basic_info["工序内容"])
 #wb4.close()
#管道支、吊架安装记录的信息需要根据实际填写，故表格不关闭


# SN4.1.1.1 给水管道及配件安装检验批质量验收记录

In [ ]:
docu_info = {"施工依据":"《建筑给水排水及供暖工程施工技术标准》ZJQ08-SGJB242-2017",
             "验收依据":"设计文件和《自动喷水灭火系统施工及验收规范》GB50261-2017",
            }

In [ ]:
wb5 = xw.Book(str(folder_path +'\\标准表格\\SN4.1.1.1 给水管道及配件安装检验批质量验收记录.xls'))
inspection_lot = wb5.sheets[0]
original_record = wb5.sheets[1]
#

inspection_lot.range('N4').value = basic_info["工程名称"]
inspection_lot.range('AV4').value = basic_info["分部工程"]
inspection_lot.range('BU4').value = basic_info["分项工程"]
inspection_lot.range('N5').value = basic_info["施工单位"]
inspection_lot.range('AV5').value = basic_info["项目经理"]
inspection_lot.range('N6').value = "/"#分包单位名称
inspection_lot.range('AV6').value = "/"#分包项目负责人
inspection_lot.range('BU5').value = basic_info["检验批容量"]
inspection_lot.range('BU6').value = basic_info["工序内容"]
inspection_lot.range('N7').value = docu_info["施工依据"]
inspection_lot.range('AV7').value = docu_info["验收依据"]

#给水管道水压试验设计要求及规范规定
inspection_lot.range('X9').value="设计要求：试验压力%.2fMPa" % basic_info["强度试验压力"]

#直埋金属给水管道防腐
corrosion_prevention = "/"
inspection_lot.range('BD19').value = corrosion_prevention


#给水引入管与排水排出管的水平净距不得小于1m。
inlet_outlet = np.random.randint(10,21)/10  #≥1
#室内给水与排水管道平行敷设时，两管间的最小水平净距不得小于0.5m；
parallel_laying = np.random.randint(5,16)/10 #≥0.5
#交叉铺设时，垂直净距不得小于0.15m。
cross_laying = np.random.randint(15,31)/100 #≥0.15
#给水管必须铺在排水管下面时，给水管应加套管，其长度不得小于排水管管道径的3倍
casing_pipe = "上"#否则加套管

inspection_lot.range('BD22').value ="1.给水引入管与排水排出管的水平净距（{}）米；\n2.室内给水与排水管道：平行敷设时两管间的最小水平净距（{}）米；交叉铺设时垂直净距（{}）米；\n3.给水管铺在排水管（{}）面。".format(inlet_outlet,parallel_laying,cross_laying,casing_pipe) 
original_record.range('AC10').value ="1.给水引入管与排水排出管的水平净距{}米；\n2.室内给水与排水管道：平行敷设时两管间的最小水平净距{}米；交叉铺设时垂直净距{}米；\n3.给水管铺在排水管{}面。".format(inlet_outlet,parallel_laying,cross_laying,casing_pipe)

#金属给水管道及管件焊接
inspection_lot.range('BD27').value ="DN50及以下管道采用螺纹连接，DN50以上管道采用沟槽连接，无焊接管道"
original_record.range('AC11').value = "DN50及以下管道采用螺纹连接，DN50以上管道采用沟槽连接，无焊接管道"

#给水水平管道应有2‰～5‰的坡度坡向泄水装置。
pipe_slope = "给水水平管道的坡度：{}‰\n坡向：泄水装置".format(np.random.randint(2,5+1))
inspection_lot.range('BD31').value = pipe_slope
original_record.range('AC12').value = pipe_slope

#管道支吊架
inspection_lot.range('BD35').value ="管道支吊架的安装：平整牢固\n其间距符合图纸和规范要求"
inspection_lot.range('AC13').value ="管道支吊架的安装：平整牢固\n其间距符合图纸和规范要求"

#一般项目
inspection_lot.range('BD46').value ="总测点数：20，合格点数：20。合格点率：100%。"

#材料规定
inspection_lot.range('BD61').value ="经检查：管道采用的管件与管材相适应，阀门水压试验合格。符合要求"

#给水管道连接处渗漏问题防治
inspection_lot.range('BD63').value ="经检查：镀锌类管道未使用焊接连接，管件管材无混用，管道水压试验合格。符合要求"



if basic_info["是否隐蔽"] == "是":
    wb5.save(str(folder_path+"\\工序报验\\%s - 06检验批验收记录.xls") % basic_info["工序内容"])
else:
        wb5.save(str(folder_path+"\\工序报验\\%s - 05检验批验收记录.xls") % basic_info["工序内容"])
 #wb5.close()


#检验批原始资料中的数据需要根据实际填写，故表格不关闭